# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0


# strategies

In [3]:
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
    from google.colab import auth
    # Authenticates the Colab machine and also the TPU using your
    # credentials so that they can access your private GCS buckets.
    auth.authenticate_user()

In [4]:
# Detect hardware
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
    tpu = None
    gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy
if tpu:
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu, steps_per_run=128) # Going back and forth between TPU and host is expensive. Better to run 128 batches on the TPU before reporting back.
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
elif len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
    print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    print('Running on single GPU ', gpus[0].name)
else:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

Running on single GPU  /device:GPU:0
Number of accelerators:  1


In [5]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

C:\Users\Khalid\Documents\projects\Generative_Models


# Dataset loading

In [6]:
dataset_name='pokemon'

In [7]:
images_dir = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
validation_percentage = 20
valid_format = 'png'

In [8]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [9]:
with strategy.scope():
    imgs_list = create_image_lists(
        image_dir=images_dir, 
        validation_pct=validation_percentage, 
        valid_imgae_formats=valid_format
    )

  DEBUG    | Looking for images in 'cat01'


  INFO     | 34 file found


  DEBUG    | Looking for images in 'cat02'


  INFO     | 775 file found


In [10]:
inputs_shape= image_size=(200, 200, 3)
batch_size = 32//2
latents_dim = 32
intermediate_dim = 50

In [11]:
with strategy.scope():
    training_generator, testing_generator = get_generators(
        images_list=imgs_list, 
        image_dir=images_dir, 
        image_size=image_size, 
        batch_size=batch_size, 
        class_mode=None
    )

  INFO     | Found 662 training files


  INFO     | Found 147 validation files


In [12]:
import tensorflow as tf

In [13]:
with strategy.scope():
    train_ds = tf.data.TFRecordDataset.from_generator(
        lambda: training_generator, 
        output_types=tf.float32 ,
        output_shapes=tf.TensorShape((batch_size, ) + image_size)
    )

    test_ds = tf.data.TFRecordDataset.from_generator(
        lambda: testing_generator, 
        output_types=tf.float32 ,
        output_shapes=tf.TensorShape((batch_size, ) + image_size)
    )


In [14]:
#highest = tf.map_fn(lambda x : (-x, x), indices, dtype=(tf.int32, tf.int32))

#map(tuple, csvreader), (tf.uint8,) * (28 ** 2))

In [15]:
_instance_scale=1.0
#for data in train_ds:
#    _instance_scale = float(data[0].numpy().max())
#    break

In [16]:
_instance_scale

1.0

In [17]:
import numpy as np
from collections.abc import Iterable

In [18]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [19]:
_outputs_shape

120000

# Model's Layers definition

In [20]:
with strategy.scope():
    menc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
                tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(units=latents_dim)]

    venc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
                tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(units=latents_dim)]

    dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
                tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
                tf.keras.layers.Dense(units=_outputs_shape),
                tf.keras.layers.Reshape(inputs_shape)]

In [21]:
inputs_shape

(200, 200, 3)

# Model definition

In [22]:
model_name = dataset_name+'VAE_Dense_reconst_ell'
experiments_dir='experiments'+sep_local+model_name

In [23]:
from training.autoencoding_basic.autoencoders.VAE import VAE as AE

In [24]:
inputs_shape=image_size

In [25]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': menc_lays
    },
    
    {
        'name': 'inference_logvariance', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': venc_lays
    },
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [26]:
from utils.data_and_files.file_utils import create_if_not_exist

In [27]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [28]:
create_if_not_exist(_restore)
_restore

'experiments\\pokemonVAE_Dense_reconst_ell\\var_save_dir'

In [29]:
#to restore trained model, set filepath=_restore

In [30]:
with strategy.scope():
    ae = AE( 
        name=model_name,
        latents_dim=latents_dim,
        batch_size=batch_size,
        variables_params=variables_params, 
        filepath=None
        )

Model: "inference_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_mean_inputs (Input [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 200, 200, 25)      100       
_________________________________________________________________
dense_1 (Dense)              (None, 200, 200, 25)      650       
_________________________________________________________________
flatten (Flatten)            (None, 1000000)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                32000032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)             

  WARNING  | None


Model: "inference_logvariance"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_logvariance_inputs [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense_3 (Dense)              (None, 200, 200, 25)      100       
_________________________________________________________________
dense_4 (Dense)              (None, 200, 200, 25)      650       
_________________________________________________________________
flatten_1 (Flatten)          (None, 1000000)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                32000032  
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)      

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 50)                1650      
_________________________________________________________________
dense_8 (Dense)              (None, 120000)            6120000   
_________________________________________________________________
reshape (Reshape)            (None, 200, 200, 3)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 200, 200, 3)       12        
_________________________________________________________________
dropout_2 (Dropout)          (None, 200, 200, 3)       0

  WARNING  | None


In [31]:
with strategy.scope():
    ae.compile(metrics=None)

Model: "pokemonVAE_Dense_reconst_ell"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inference_logvariance_inputs (I [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
inference_logvariance (Model)   (None, 32)           32000910    inference_logvariance_inputs[0][0
__________________________________________________________________________________________________
tf_op_layer_Mul (TensorFlowOpLa [(None, 32)]         0           inference_logvariance[1][0]      
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 32)]         0           tf_op_layer_Mul[0][0]            
_______________________________________________________________________

# Callbacks

In [32]:

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [33]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=12, 
    verbose=1, 
    restore_best_weights=False
)

In [34]:
ms = ModelSaver(filepath=_restore)

In [35]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'experiments\\pokemonVAE_Dense_reconst_ell\\csv_dir\\pokemonVAE_Dense_reconst_ell.csv'

In [36]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [37]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [38]:
import numpy as np

# Model Training

In [41]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=5,#int(1e4),
    epochs=5,#int(1e6), 
    verbose=1,#2,
    #callbacks=[ es, ms, csv_log, sg],
    #workers=-1,
    #use_multiprocessing=True,
    #validation_data=test_ds,
    #validation_steps=5,#int(1e4)
)

Epoch 1/5
5/5 [==============================] - ETA: 0s - loss: 60503.4844 - x_logits_loss: 60502.4688 - x_logpdf_loss: -1.4256 - z_latents_loss: 2.348 - ETA: 0s - loss: 60592.0078 - x_logits_loss: 60590.7969 - x_logpdf_loss: -1.4230 - z_latents_loss: 2.519 - ETA: 0s - loss: 60740.0742 - x_logits_loss: 60738.6992 - x_logpdf_loss: -1.4160 - z_latents_loss: 2.654 - ETA: 0s - loss: 59311.3164 - x_logits_loss: 59309.9609 - x_logpdf_loss: -1.4185 - z_latents_loss: 2.631 - ETA: 0s - loss: 58729.8984 - x_logits_loss: 58728.5625 - x_logpdf_loss: -1.4148 - z_latents_loss: 2.600 - 1s 158ms/step - loss: 58729.8984 - x_logits_loss: 58728.5625 - x_logpdf_loss: -1.4148 - z_latents_loss: 2.6009
Epoch 2/5
5/5 [==============================] - ETA: 0s - loss: 59492.4531 - x_logits_loss: 59491.1484 - x_logpdf_loss: -1.3873 - z_latents_loss: 2.499 - ETA: 0s - loss: 56995.8516 - x_logits_loss: 56994.7227 - x_logpdf_loss: -1.3973 - z_latents_loss: 2.340 - ETA: 0s - loss: 56494.9805 - x_logits_loss: 56493

# Model Evaluation

## inception_score

In [42]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [43]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

calculating the inception_score mean ...


KeyboardInterrupt: 

## Frechet_inception_distance

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [ ]:
from training.generators.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)